# Import necessory Libraries

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import wordnet
import pickle

# Data Preprocessing Function

In [ ]:
def DataPreprocess(data):
    is_noun = lambda pos: pos[:2] == 'NN'

    def removespecial(s):
        r=list(s)
        for i in range(len(r)):
            if not(r[i].isalpha() or r[i].isdigit()):
                r[i]=' '
        return "".join(r)

    def preprocess(s):
        t=s.split()
        i=0
        while i<len(t):
            if len(t[i])==1 and t[i] in ['s','t']:
                t[i-1]+=t[i]
                t.pop(i)
            else:
                i+=1
        return " ".join(t)

    def removesingle(s):
        t=s.split()
        i=0
        while i<len(t):
            t[i]=t[i].lower()
            if len(t[i])==1 and t[i] not in ['i','a']:
                t.pop(i)
            else:
                i+=1
        return " ".join(t)

    nouns_title=[]
    nouns_content=[]

    for i in data.index:  

        try:
            s=removespecial(data['Content'][i])
            s=preprocess(s)
            s=removesingle(s)
            tokenized = nltk.word_tokenize(s)
            nouns1 = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)]


            s=removespecial(data['Title'][i])
            s=preprocess(s)
            s=removesingle(s)
            tokenized = nltk.word_tokenize(s)
            nouns2 = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)]

            nouns_content.append(nouns1)
            nouns_title.append(nouns2)
        except:
            pass
    return nouns_title,nouns_content

# find Context Words

In [ ]:
def findContextWords(nouns_title,nouns_content):
  context_words=[]
  for i in range(len(nouns_content)):
      sim=[]
      nouns=nouns_title[i]
      if len(nouns)==0:
          pass
      else:
          for j in nouns_content[i]:
              s=0
              for n in nouns:
                  syn1 = wordnet.synsets(j)
                  syn2 = wordnet.synsets(n)
                  for s1 in syn1:
                      for s2 in syn2:
                          if s1.wup_similarity(s2)!=None:
                              s+=s1.wup_similarity(s2)
              if s/len(nouns)>=1:
                  context_words.append(j)
      print(i+1)
  return context_words

# Trigram Function

In [ ]:
def build_trigram(context_words):
  trigram={}
  for i in range(len(context_words)-2):
    w1,w2,w3=context_words[i:i+3]
    trigram[(w1,w2,w3)]=trigram.get((w1,w2,w3),0)+1
    
  return trigram

# To build trigrams

### V1

In [ ]:
# context_words is a list of context words derived in preprocessing stage
with open('Context and Ngram/V1/context_words_terror.pickle', 'rb') as f:
    terror=pickle.load(f)

terror=build_trigram(terror)

with open('Context and Ngram/V1/context_words_crime.pickle', 'rb') as f:
    crime=pickle.load(f)

crime=build_trigram(crime)

with open('Context and Ngram/V1/context_words_cancer.pickle', 'rb') as f:
    cancer=pickle.load(f)

cancer=build_trigram(cancer)

with open('Context and Ngram/V1/context_words_health.pickle', 'rb') as f:
    health=pickle.load(f)

health=build_trigram(health)

### For V2

In [ ]:
# context_words is a list of context words derived in preprocessing stage
with open('Context and Ngram/V2/context_words_trainterror.pickle', 'rb') as f:
    terror=pickle.load(f)

terror=build_trigram(terror)

with open('Context and Ngram/V2/context_words_traincrime.pickle', 'rb') as f:
    crime=pickle.load(f)

crime=build_trigram(crime)

with open('Context and Ngram/V2/context_words_traincancer.pickle', 'rb') as f:
    cancer=pickle.load(f)

cancer=build_trigram(cancer)

with open('Context and Ngram/V2/context_words_trainhealth.pickle', 'rb') as f:
    health=pickle.load(f)

health=build_trigram(health)

# Load Previously build trigrams data

### V1

In [ ]:
with open('Context and Ngram/V1/trigram_cancer.pickle', 'rb') as f:
    cancer=pickle.load(f)
with open('Context and Ngram/V1/trigram_crime.pickle', 'rb') as f:
    crime=pickle.load(f)
with open('Context and Ngram/V1/trigram_terror.pickle', 'rb') as f:
    terror=pickle.load(f)
with open('Context and Ngram/V1/trigram_health.pickle', 'rb') as f:
    health=pickle.load(f)

### V2

In [ ]:
with open('Context and Ngram/V2/trigram_traincancer.pickle', 'rb') as f:
    cancer=pickle.load(f)
with open('Context and Ngram/V2/trigram_traincrime.pickle', 'rb') as f:
    crime=pickle.load(f)
with open('Context and Ngram/V2/trigram_trainterror.pickle', 'rb') as f:
    terror=pickle.load(f)
with open('Context and Ngram/V2/trigram_trainhealth.pickle', 'rb') as f:
    health=pickle.load(f)

# Training and Validation

### BW and CW

In [ ]:
preds=[]
actual=[]
for t in ['crime','cancer','terror','health']:
    
    data=pd.read_csv('Validation Data/test'+t+'.csv')
    
    nouns_title,nouns_content=DataPreprocess(data)
    
    same=0
    for i in range(len(nouns_title)):
        a=nouns_title[i]
        b=nouns_content[i]

        def contextwords(a,b):
            context_words=[]
            for j in b:
                s=0
                for n in a:
                    syn1 = wordnet.synsets(j)
                    syn2 = wordnet.synsets(n)
                    for s1 in syn1:
                        for s2 in syn2:
                            if s1.wup_similarity(s2)!=None:
                                s+=s1.wup_similarity(s2)
                if s/len(a)>=1:
                    context_words.append(j)
            return context_words

        context_words=contextwords(a,b)

        bigram=build_trigram(context_words)

        article_type={1:'crime',2:'cancer',3:'terror',4:'health'}

        cw=[]
        iter=1
        for j in [crime,cancer,terror,health]:
            c=0
            bw=[]
            for x in bigram:
                if j.get(x,0):
                    bw.append(bigram[x]*j[x])
                    c+=1
            if len(bw)==0:
                cw.append((iter,0))
            else:
                f=0
                mx=0
                for x in bw:
                    s=np.log(1+(c*x))
                    if s>mx:
                        mx=s
                cw.append((iter,mx))
            iter+=1


        ans=max(cw,key=lambda x:x[1])
        
        preds.append(article_type[ans[0]])
        actual.append(t)
        
    print(t,'Completed')  
            

Results V1

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(actual,preds,average='weighted')

Results V2

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(actual,preds,average='weighted')

### KESALJ

In [ ]:
preds1=[]
actual1=[]
for t in ['crime','cancer','terror','health']:
    
    data=pd.read_csv('Validation Data/test'+t+'.csv')
    
    nouns_title,nouns_content=DataPreprocess(data)
    
    same=0
    for i in range(len(nouns_title)):
        a=nouns_title[i]
        b=nouns_content[i]

        def contextwords(a,b):
            context_words=[]
            for j in b:
                s=0
                for n in a:
                    syn1 = wordnet.synsets(j)
                    syn2 = wordnet.synsets(n)
                    for s1 in syn1:
                        for s2 in syn2:
                            if s1.wup_similarity(s2)!=None:
                                s+=s1.wup_similarity(s2)
                if s/len(a)>=1:
                    context_words.append(j)
            return context_words

        context_words=contextwords(a,b)

        bigram=build_trigram(context_words)

        article_type={1:'crime',2:'cancer',3:'terror',4:'health'}
        iter=1
        kjd=[]
        for j in [crime,cancer,terror,health]:
            #bigrams=set(list(bigram.keys())+list(j.keys()))
            bigrams=list(set(bigram.keys()) & set(j.keys()))
        
            dis=0
            for x in bigrams:
                a=bigram.get(x,0)
                b=j.get(x,0)
                dis+=(2*(b-a)/(a+b))
            kjd.append((iter,dis))
            iter+=1
        
        ans=max(kjd,key=lambda x:x[1])
        #print(kjd)
        preds1.append(article_type[ans[0]])
        actual1.append(t) 
    

Results V1

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(actual1,preds1,average='weighted')

Results V2

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(actual1,preds1,average='weighted')

# Final Test

### BW and CW

In [ ]:
preds=[]
actual=[]

data=pd.read_csv('Test Data/Finaltest.csv')

nouns_title,nouns_content=DataPreprocess(data)
actual=list(data['Query Name'])

for i in range(len(nouns_title)):
    a=nouns_title[i]
    b=nouns_content[i]

    def contextwords(a,b):
        context_words=[]
        for j in b:
            s=0
            for n in a:
                syn1 = wordnet.synsets(j)
                syn2 = wordnet.synsets(n)
                for s1 in syn1:
                    for s2 in syn2:
                        if s1.wup_similarity(s2)!=None:
                            s+=s1.wup_similarity(s2)
            if len(a) and s/len(a)>=1:
                context_words.append(j)
        return context_words

    context_words=contextwords(a,b)

    bigram=build_trigram(context_words)

    article_type={1:'crime',2:'cancer',3:'terror',4:'health'}

    cw=[]
    iter=1
    for j in [crime,cancer,terror,health]:
        c=0
        bw=[]
        for x in bigram:
            if j.get(x,0):
                bw.append(bigram[x]*j[x])
                c+=1
        if len(bw)==0:
            cw.append((iter,0))
        else:
            f=0
            mx=0
            for x in bw:
                s=np.log(1+(c*x))
                if s>mx:
                    mx=s
            cw.append((iter,mx))
        iter+=1


    ans=max(cw,key=lambda x:x[1])

    preds.append(article_type[ans[0]])
print('Completed')

Results V1

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(actual,preds,average='weighted')

Results V2

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(actual,preds,average='weighted')

### KESALJ

In [ ]:
preds1=[]
actual1=[]

data=pd.read_csv('Test Data/Finaltest.csv')
actual1=list(data['Query Name'])

nouns_title,nouns_content=DataPreprocess(data)

for i in range(len(nouns_title)):
    a=nouns_title[i]
    b=nouns_content[i]

    def contextwords(a,b):
        context_words=[]
        for j in b:
            s=0
            for n in a:
                syn1 = wordnet.synsets(j)
                syn2 = wordnet.synsets(n)
                for s1 in syn1:
                    for s2 in syn2:
                        if s1.wup_similarity(s2)!=None:
                            s+=s1.wup_similarity(s2)
            if len(a) and s/len(a)>=1:
                context_words.append(j)
        return context_words

    context_words=contextwords(a,b)

    bigram=build_trigram(context_words)

    article_type={1:'crime',2:'cancer',3:'terror',4:'health'}
    iter=1
    kjd=[]
    for j in [crime,cancer,terror,health]:
        #bigrams=set(list(bigram.keys())+list(j.keys()))
        bigrams=list(set(bigram.keys()) & set(j.keys()))
        dis=0
        for x in bigrams:
            a=bigram.get(x,0)
            b=j.get(x,0)
            dis+=(2*(b-a)/(a+b))
        kjd.append((iter,dis))
        iter+=1

    ans=max(kjd,key=lambda x:x[1])

    preds1.append(article_type[ans[0]])
print('Completed')
        

Results V1

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(actual1,preds1,average='weighted')

Results V2

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(actual1,preds1,average='weighted')